In [12]:
# modules and tools

import pathlib
import re
from collections import defaultdict
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.renderers.default; "vscode"
pio.renderers


# User settings

target_dir = pathlib.Path("/Users/will/Dropbox/zettelkasten/")
csv_file = '/Users/will/Dropbox/Projects/Dashboard/zkstats/stats.csv'
UUID_sign = "›"


# functions

def append_new_line(filename, text_to_append):
    # Append given text as a new line at the end of file
    # Open the file in append & read mode ('a+')
    with open(filename, "a+") as file_object:
        # Move read cursor to the start of file.
        file_object.seek(0)
        # If file is not empty then append '\n'
        data = file_object.read(100)
        if len(data) > 0:
            file_object.write("\n")
        # Append text at the end of file
        file_object.write(text_to_append)

# Initialize counters


tw = 0
tl = 0
tz = 0

# Regex

date_pattern = re.compile(r"\d{8}")
link_pattern = re.compile(r"((?<!{UUID_sign)\[\[\d{8})")

# Initalize output csv with headers

f = open(csv_file, "w+")
append_new_line(csv_file, "date,tz,tw,tl")

# Files is a dictionary mapping of a date to the list of files with that date

files = defaultdict(list)
for child in target_dir.iterdir():
    # Skip directories
    if child.is_dir():
        continue
    match = date_pattern.search(child.name)
    # Skip files that do not match the date pattern
    if match is None:
        continue
    file_date = match.group()
    files[file_date].append(child)

for date in sorted(files):

    for filename in files[date]:

         # Word Count
        file = open(filename, "r")
        data = file.read()
        words = data.split()
        w = len(words)
        tw += w

        # Link Count
        l = len(re.findall(link_pattern, data))
        tl += l

        # Zettel Count
        tz += 1

        # print(date,tz, tw, tl, w, l) # For troubleshooting
        # Set date range. Remove this line for totals.
        if date.startswith("20"):
            append_new_line(csv_file, f"{date},{tz},{tw},{tl}")
            file.close()

In [18]:
import csv

# Read the csv and convert it into a list of dictionaries
with open(csv_file) as pkm:
    reader = csv.reader(pkm)
    
    # use of next to skip first title row in csv file
    next(reader) 
    date = []
    tz = []
    tw = []
    tl = []


    # in this loop I did sum of column 1 which is revenue in csv file and counted total months which is column 0 
    for row in reader:

        tz.append(float(row[1]))
        tw.append(float(row[2]))
        tl.append(float(row[3]))
        date.append(row[0])

    zpd = round((tz)[-1]-(tz)[1])
    wpd = round((tw)[-1]-(tw)[1])
    lpd = round((tl)[-1]-(tl)[1])
    zettelkasting = (len(tz)-1)

output = f""" 
Zettelkasten Analysis
Session or days on which some zettelkasting took place: {zettelkasting}
{'-'*40}
Total Zettel: {round((tz)[-1]-(tz)[1])}
    Average # of Zettel per session: {round((zpd/zettelkasting))}
Total Words: {round((tw)[-1]-(tw)[1])} 
    Average # of Words per zettel: {round(((tw)[-1]-(tw)[1])/((tz)[-1]-(tz)[1]))}
Total Links: {round((tl)[-1]-(tl)[1])}
    Average # of Links per zettel: {round(((tl)[-1]-(tl)[1])/((tz)[-1]-(tz)[1]))}
"""
print(output)

 
Zettelkasten Analysis
Session or days on which some zettelkasting took place: 2556
----------------------------------------
Total Zettel: 2555
    Average # of Zettel per session: 1
Total Words: 576570 
    Average # of Words per zettel: 226
Total Links: 12296
    Average # of Links per zettel: 5



## Make Word Count Graph

In [14]:
zk = pd.read_csv(csv_file)

zk['date'] = pd.to_datetime(zk['date'], format='%Y%m%d', errors='coerce')

fig = px.line(zk, x=zk['date'], y=zk['tw'], 
              title='My Total Zettelkasting Progress',
              labels=dict(date="Word Count", tw="Total Words",
              plot_bgcolor='rgb(200,200,200)',
              showlegend=True)
             )
fig.update_xaxes(title_font=dict(size=18, family='Courier', color='crimson'))
fig.update_yaxes(title_font=dict(size=18, family='Courier', color='crimson'))
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

fig.show()


## Make Link Count Graph

In [15]:
zk = pd.read_csv(csv_file)

zk['date'] = pd.to_datetime(zk['date'], format='%Y%m%d', errors='coerce')

fig = px.line(zk, x=zk['date'], y=zk['tl'], 
              title='My Total Zettelkasting Progress',
              labels=dict(date="Link Count" , tl="Total Links",
              plot_bgcolor='rgb(200,200,200)',
              showlegend=True)
             )
fig.update_xaxes(title_font=dict(size=18, family='Courier', color='crimson'))
fig.update_yaxes(title_font=dict(size=18, family='Courier', color='crimson'))
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)



fig.show()

## Make Zettel Count Graph

In [16]:
zk = pd.read_csv(csv_file)

zk['date'] = pd.to_datetime(zk['date'], format='%Y%m%d', errors='coerce')

fig = px.line(zk, x=zk['date'], y=zk['tz'], 
              title='My Total Zettelkasting Progress',
              labels=dict(date="Zettel Count", tz="Total Zettel",
              plot_bgcolor='rgb(200,200,200)',
              showlegend=True)
             )
fig.update_xaxes(title_font=dict(size=18, family='Courier', color='crimson'))
fig.update_yaxes(title_font=dict(size=18, family='Courier', color='crimson'))
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)


fig.show()